In [ ]:
import pandas as pd  

# 读取CSV文件  
file_path = 'RESSET_DRESSTK_2021__1.csv'  # 替换为你的CSV文件路径  
df = pd.read_csv(file_path)  

# 删除包含缺失数据的行  
df_clean = df.dropna()  

# 将清理后的数据保存到一个新的CSV文件  
output_file_path = 'cleaned_file_1.csv'  # 替换为你想要保存的文件路径  
df_clean.to_csv(output_file_path, index=False)  

print("清理完成，缺失数据的行已被删除。")

In [ ]:
filepath = 'cleaned_file_1.csv'
data = pd.read_csv(filepath)
data = data.sort_values('Date')
data.head()

In [ ]:
def feature_project(data):
    # TODO 请在此处添加特征工程代码,返回特征列表
    features=['Clpr', 'Trdvol', 'Trdsum', 'Dret','PE']
    return features

In [ ]:
features = feature_project(data)
price = data[features]
price


In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler=[]
for i in range(len(features)):
    scaler_tmp=MinMaxScaler(feature_range=(-1, 1))
    scaler.append(scaler_tmp)
    price[features[i]] = scaler[i].fit_transform(price[features[i]].values.reshape(-1,1))
price.head()


In [ ]:
#数据集
import numpy as np
def split_data(stock, lookback):
    data_raw = stock.to_numpy() 
    print('data_raw.shape = ',data_raw.shape)
    data = []
    
    # you can free play（seq_length）
    for index in range(len(data_raw) - lookback): 
        data.append(data_raw[index: index + lookback])
    
    data = np.array(data);
    print('data.shape = ',data.shape)
    test_set_size = int(np.round(0.2 * data.shape[0]))
    train_set_size = data.shape[0] - (test_set_size)
    
    x_train = data[:train_set_size,:-1,:]
    #print(x_train)
    y_train = data[:train_set_size,-1,:]
    
    x_test = data[train_set_size:,:-1,:]
    y_test = data[train_set_size:,-1,:]
    
    return [x_train, y_train, x_test, y_test]

In [ ]:
lookback = 100
x_train0=[]
y_train0=[]
x_test0=[]
y_test0=[]
x_train1, y_train1, x_test1, y_test1 = split_data(price, lookback)
x_train0.append(x_train1)
y_train0.append(y_train1)
x_test0.append(x_test1)
y_test0.append(y_test1)

print('x_train.shape = ',x_train0[0].shape)
print('y_train.shape = ',y_train0[0].shape)
print('x_test.shape = ',x_test0[0].shape)
print('y_test.shape = ',y_test0[0].shape)


In [ ]:
import torch
import torch.nn as nn
from zmq import device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = "cpu"
x_train=[]
x_test=[]
y_train_lstm=[]
y_test_lstm=[]


x_trainn = torch.from_numpy(x_train0[0]).type(torch.Tensor)
x_testn = torch.from_numpy(x_test0[0]).type(torch.Tensor)


y_train_lstmn = torch.from_numpy(y_train0[0]).type(torch.Tensor)
y_test_lstmn = torch.from_numpy(y_test0[0]).type(torch.Tensor)

x_trainn = x_trainn.to(device)
x_testn = x_testn.to(device)
y_train_lstmn = y_train_lstmn.to(device)
y_test_lstmn = y_test_lstmn.to(device)

print('x_train.shape = ',x_trainn.shape)
print('y_train.shape = ',y_train_lstmn.shape)
print('x_test.shape = ',x_testn.shape)
print('y_test.shape = ',y_test_lstmn.shape)

x_train.append(x_trainn)
x_test.append(x_testn)
y_train_lstm.append(y_train_lstmn)
y_test_lstm.append(y_test_lstmn)


    

In [ ]:
input_dim = len(features)
hidden_dim = 32
num_layers = 2
output_dim = len(features)
num_epochs = 500

In [ ]:
# class LSTM(nn.Module):
#     def __init__(self, input_dim, hidden_dim, num_layers, output_dim):
#         super(LSTM, self).__init__()
#         self.hidden_dim = hidden_dim
#         self.num_layers = num_layers
        
#         self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
#         self.fc = nn.Linear(hidden_dim, output_dim)

#     def forward(self, x):
#         h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).requires_grad_()
#         c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).requires_grad_()
#         out, (hn, cn) = self.lstm(x, (h0.detach(), c0.detach()))
#         out = self.fc(out[:, -1, :]) 
#         return out
    
class LSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, output_dim, dropout_prob=0.2):
        super(LSTM, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc1 = nn.Linear(hidden_dim, hidden_dim*2)  # 增加线性层的大小
        self.dropout = nn.Dropout(p=dropout_prob)         # 添加Dropout层
        self.fc2 = nn.Linear(hidden_dim*2, output_dim)  # 第二个全连接层

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).requires_grad_().to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).requires_grad_().to(x.device)
        out, (hn, cn) = self.lstm(x, (h0.detach(), c0.detach()))
        out = self.fc1(out[:, -1, :])
        out = torch.relu(out)
        out = self.dropout(out)
        out = self.fc2(out)
        return out
    

In [ ]:
print(device)
model0 = LSTM(input_dim=input_dim, hidden_dim=hidden_dim, output_dim=output_dim, num_layers=num_layers).to(device)

criterion = torch.nn.MSELoss()
optimiser0 = torch.optim.Adam(model0.parameters(), lr=0.01,weight_decay=1e-5)


In [ ]:
#模型训练
import time

hist0 = np.zeros(num_epochs)
start_time = time.time()
lstm = []

for t in range(num_epochs):
    y_train_pred0 = model0(x_train[0])

    loss = criterion(y_train_pred0, y_train_lstm[0])
    print("Epoch ", t, "MSE: ", loss.item())
    hist0[t] = loss.item()

    optimiser0.zero_grad()
    loss.backward()
    optimiser0.step()
    

In [ ]:
if (device == "cpu"):
    predict = pd.DataFrame(scaler[0].inverse_transform(y_train_pred0[:,:1].detach().numpy()))
    original = pd.DataFrame(scaler[0].inverse_transform(y_train_lstm[0][:,:1].detach().numpy()))
elif (device == "cuda"):
    predict = pd.DataFrame(scaler[0].inverse_transform(y_train_pred0[:,:1].cpu().detach().numpy()))
    original = pd.DataFrame(scaler[0].inverse_transform(y_train_lstm[0][:,:1].cpu().detach().numpy()))

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.set_style("darkgrid")    

fig = plt.figure()
fig.subplots_adjust(hspace=0.2, wspace=0.2)

plt.subplot(1, 2, 1)
ax = sns.lineplot(x = original.index, y = original[0], label="Data", color='royalblue')
ax = sns.lineplot(x = predict.index, y = predict[0], label="Training Prediction (LSTM)", color='tomato')
ax.set_title('Stock price', size = 14, fontweight='bold')
ax.set_xlabel("Days", size = 14)
ax.set_ylabel("Cost (USD)", size = 14)
ax.set_xticklabels('', size=10)


plt.subplot(1, 2, 2)
ax = sns.lineplot(data=hist0, color='royalblue')
ax.set_xlabel("Epoch", size = 14)
ax.set_ylabel("Loss", size = 14)
ax.set_title("Training Loss", size = 14, fontweight='bold')
fig.set_figheight(6)
fig.set_figwidth(16)

In [ ]:

""" #test验证
 import math, time
from sklearn.metrics import mean_squared_error
 
# make predictions
y_test_pred = model(x_test)

# invert predictions
y_train_pred = scaler.inverse_transform(y_train_pred.detach().numpy())
y_train = scaler.inverse_transform(y_train_lstm.detach().numpy())
y_test_pred = scaler.inverse_transform(y_test_pred.detach().numpy())
y_test = scaler.inverse_transform(y_test_lstm.detach().numpy())

# calculate root mean squared error
trainScore = math.sqrt(mean_squared_error(y_train[:,0], y_train_pred[:,0]))
print('Train Score: %.2f RMSE' % (trainScore))
testScore = math.sqrt(mean_squared_error(y_test[:,0], y_test_pred[:,0]))
print('Test Score: %.2f RMSE' % (testScore))
lstm.append(trainScore)
lstm.append(testScore)
lstm.append(training_time) """

In [ ]:
""" # 注意这里标签索引是一个列表
new_price = price[['Clpr']]
new_price """

In [ ]:
""" # shift train predictions for plotting
trainPredictPlot = np.empty_like(new_price)
trainPredictPlot[:, 0] = np.nan
trainPredictPlot[lookback:len(y_train_pred)+lookback, :] = y_train_pred """

In [ ]:
""" # shift test predictions for plotting
testPredictPlot = np.empty_like(new_price)
testPredictPlot[:, :] = np.nan
testPredictPlot[len(y_train_pred)+lookback-1:len(price)-1, :] = y_test_pred """

In [ ]:
""" original = scaler.inverse_transform(price['Clpr'].values.reshape(-1,1))

predictions = np.append(trainPredictPlot, testPredictPlot, axis=1)
predictions = np.append(predictions, original, axis=1)
result = pd.DataFrame(predictions) """

In [ ]:
""" import plotly.express as px
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Scatter(go.Scatter(x=result.index, y=result[0],
                    mode='lines',
                    name='Train prediction')))
fig.add_trace(go.Scatter(x=result.index, y=result[1],
                    mode='lines',
                    name='Test prediction'))
fig.add_trace(go.Scatter(go.Scatter(x=result.index, y=result[2],
                    mode='lines',
                    name='Actual Value')))
fig.update_layout(
    xaxis=dict(
        showline=True,
        showgrid=True,
        showticklabels=False,
        linecolor='white',
        linewidth=2
    ),
    yaxis=dict(
        title_text='Close (USD)',
        titlefont=dict(
            family='Rockwell',
            size=12,
            color='white',
        ),
        showline=True,
        showgrid=True,
        showticklabels=True,
        linecolor='white',
        linewidth=2,
        ticks='outside',
        tickfont=dict(
            family='Rockwell',
            size=12,
            color='white',
        ),
    ),
    showlegend=True,
    template = 'plotly_dark'

)



annotations = []
annotations.append(dict(xref='paper', yref='paper', x=0.0, y=1.05,
                              xanchor='left', yanchor='bottom',
                              text='Results (LSTM)',
                              font=dict(family='Rockwell',
                                        size=26,
                                        color='white'),
                              showarrow=False))
fig.update_layout(annotations=annotations)

fig.show() """

In [ ]:
filepath = 'RESSET_DRESSTK_2021.csv'
data2 = pd.read_csv(filepath)
data2 = data2.sort_values('Date')
data2.head()
print(len(data2))

In [ ]:
price2 = data2[features]
price2.head()

In [ ]:
from sklearn.preprocessing import MinMaxScaler

for i in range(len(features)):
    price2[features[i]] = scaler[i].fit_transform(price2[features[i]].values.reshape(-1,1))

print(price2.head())


In [ ]:
#数据集
import numpy as np
def split_data1(stock, lookback):
    data_raw = stock.to_numpy() 
    #print('data_raw.shape = ',data_raw.shape)
    data = []
    
    #print(len(data_raw))
    # you can free play（seq_length）
    
    data.append(data_raw[len(data)-lookback:])
    #print(data)
    
    data = np.array(data);
    #print('data.shape = ',data.shape)
    
    x_train = data[:,:-1,:]
    #print(x_train)
    y_train = data[:,-1,:]
    
    
    return [x_train, y_train]

In [ ]:

xnow=[]
ynow=[]
x_now1, y_now1  = split_data1(price2, lookback)
xnow.append(x_now1)
ynow.append(y_now1)

print("xnow.shape:",xnow[0].shape)
print("ynow.shape:",ynow[0].shape)
# print(len(xnow[0]))
# print(xnow[0])

In [ ]:
import torch
import torch.nn as nn

x_now=[]
y_now_lstm=[]

x_trainn = torch.from_numpy(xnow[0]).type(torch.Tensor).to(device)
y_train_lstmn = torch.from_numpy(ynow[0]).type(torch.Tensor).to(device)

print('x_train.shape = ',x_trainn.shape)
print('y_train.shape = ',y_train_lstmn.shape)

x_now.append(x_trainn)
y_now_lstm.append(y_train_lstmn)

    


In [ ]:
import torch 
from sklearn.preprocessing import MinMaxScaler

pdtime = 100
pdclpr= []

price1 = data2[features]


for i in range (pdtime):
    y_result = model0(x_now[0])

    Clpr1 = scaler[0].inverse_transform(y_result[:,0:1].cpu().detach().numpy())
    
    pdclpr.append(Clpr1[0][0])

    # print(y_result[0][0])
    
    price2 = price2.drop(price2.index[0])
    
    
    new_row_data = {}
    for i in range(len(features)):
        new_row_data[features[i]] = y_result[0][i]
    # print(new_row_data)

    # 假设 new_row_data 是一个字典，包含要添加的新行的数据  
    new_row = pd.DataFrame([new_row_data]) 
    

    # 使用 concat 方法添加新行  
    price2 = pd.concat([price2, new_row], ignore_index=True)

    xnow=[]
    ynow=[]
    x_now1, y_now1  = split_data1(price2, lookback)
    xnow.append(x_now1)
    ynow.append(y_now1)

    x_now=[]
    y_now_lstm=[]
    # print(xnow[0])

    
    x_trainn = torch.from_numpy(xnow[0].astype(np.float64)).type(torch.Tensor).to(device)
    y_train_lstmn = torch.from_numpy(ynow[0].astype(np.float64)).type(torch.Tensor).to(device)

    x_now.append(x_trainn)
    y_now_lstm.append(y_train_lstmn)




In [ ]:
#result
import matplotlib.pyplot as plt
import pandas as pd  

# 读取CSV文件，假设文件名为 'your_file.csv'  
df = pd.read_csv('RESSET_DRESSTK_2021__1really.csv')  

# 提取Clpr列并转换为列表  
clpr_list = df['Clpr'].tolist() 

len_min=min(len(clpr_list),len(pdclpr))
x1 = range(len_min)


# 创建一个新的图形  
plt.figure()  

# 绘制第一个列表  
plt.plot(x1, clpr_list[:len_min], label='real')  

# 绘制第二个列表  
plt.plot(x1, pdclpr, label='predict')  

# 添加图例  
plt.legend()  

# 添加标题和标签  
plt.title('My Plot with Two Lists')  
plt.xlabel('X Axis Label')  
plt.ylabel('Y Axis Label')  


# 显示图形  
plt.show()
